## Multi-Head Attention Breakdown

In [2]:
import torch
import torch.nn as nn

### Creating Batches

In [3]:
inputs = torch.nn.Embedding(4,8)
inputs = inputs.weight.data
batches = torch.stack((inputs, inputs), dim=0)


In [4]:
batches

tensor([[[-0.3816, -0.1771, -0.3567, -0.9938,  0.2858, -0.7467,  0.9492,
           1.1943],
         [ 0.6929, -0.7397,  1.4993,  0.4912, -0.2721,  1.3577,  1.2200,
           1.0504],
         [-0.8440, -1.6643, -0.3135,  1.4853,  0.0741, -0.5431,  0.9064,
          -0.3066],
         [ 1.0305, -0.9251, -1.8062, -0.2235, -0.1915, -1.6031,  0.3357,
          -1.5593]],

        [[-0.3816, -0.1771, -0.3567, -0.9938,  0.2858, -0.7467,  0.9492,
           1.1943],
         [ 0.6929, -0.7397,  1.4993,  0.4912, -0.2721,  1.3577,  1.2200,
           1.0504],
         [-0.8440, -1.6643, -0.3135,  1.4853,  0.0741, -0.5431,  0.9064,
          -0.3066],
         [ 1.0305, -0.9251, -1.8062, -0.2235, -0.1915, -1.6031,  0.3357,
          -1.5593]]])